In [1]:
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import numpy as np
import html5lib
import requests
from openpyxl import Workbook
from datetime import datetime
import webbrowser
import os


def parse_and_trim(content, content_type):
    if content_type == 'HTML':
        soup = BeautifulSoup(content, 'html.parser')
    else:
        soup = BeautifulSoup(content, 'html.parser')

    # Remove attributes from all tags
    for tag in soup.recursiveChildGenerator():
        try:
            tag.attrs = None
        except AttributeError:
            pass

    # Remove line breaks
    for linebreak in soup.find_all('br'):
        linebreak.extract()
    return soup

In [2]:
headers = {
    'User-Agent': 'GOLUB CAPITAL BDC, Inc.'
}
filing_links = pd.read_excel(
    "../GBDC__sec_filing_links.xlsx")
date_columns = ['Filing date', 'Reporting date']
for col in date_columns:
    filing_links[col] = pd.to_datetime(filing_links[col], format='%B-%d-%Y')
# for col in date_columns:
#     filing_links[col] = filing_links[col].dt.strftime("%B %d, %Y")

filing_links.head(5)

,Form type,Form description,Filing date,Reporting date,Act,Film number,Accession number,Filings URL
0,10-Q,Quarterly report [Sections 13 or 15(d)],2024-02-05,2023-12-31,34,24595836,0001476765-24-000016,https://www.sec.gov/Archives/edgar/data/147676...
1,10-K,"Annual report [Section 13 and 15(d), not S-K I...",2023-11-20,2023-09-30,34,231422817,0001476765-23-000077,https://www.sec.gov/Archives/edgar/data/147676...
2,10-Q,Quarterly report [Sections 13 or 15(d)],2023-08-07,2023-06-30,34,231147650,0001476765-23-000063,https://www.sec.gov/Archives/edgar/data/147676...
3,10-Q,Quarterly report [Sections 13 or 15(d)],2023-05-08,2023-03-31,34,23897744,0001476765-23-000042,https://www.sec.gov/Archives/edgar/data/147676...
4,10-Q,Quarterly report [Sections 13 or 15(d)],2023-02-08,2022-12-31,34,23599166,0001476765-23-000018,https://www.sec.gov/Archives/edgar/data/147676...


In [ ]:
filing_links.info()

In [ ]:
# this one is the OG one

def extract_tables(soup_content, qtr_date):
    master_table = None
    consolidated_schedule_regex = re.compile(
        r'(?i)^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    date_regex_pattern1 = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'
    for tag in soup_content.find_all(text=re.compile(consolidated_schedule_regex)):
        date_str = re.search(date_regex_pattern1, tag.find_next().text)
        if date_str is None:
            date_str = re.search(date_regex_pattern1, tag.next.text)
        print(date_str)
        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)
            qtr_date_cleaned = qtr_date.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            date_str_cleaned = date_str.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            print(qtr_date_cleaned, date_str_cleaned)

            if qtr_date_cleaned == date_str_cleaned:
                html_table = tag.find_next('table')
                new_table = pd.read_html(
                    html_table.prettify(), skiprows=0, flavor='bs4')[0]
                new_table = new_table.applymap(lambda x: unicodedata.normalize(
                    'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                new_table = new_table.replace(
                    r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                new_table = new_table.dropna(how='all', axis=0)

                if master_table is None:
                    master_table = new_table
                else:
                    master_table = pd.concat(
                        [master_table, new_table], ignore_index=True)

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    print(master_table.shape)

    return master_table


path = '/Users/fuadhassan/Desktop/BDC_RA/GBDC/GBDC_Investment.xlsx'
writer = pd.ExcelWriter(path, engine='openpyxl')
for qtr_date, html_link in zip(filing_links['Reporting date'], filing_links['Filings URL']):
    print(html_link, qtr_date)
    response = requests.get(html_link, headers=headers)
    content = parse_and_trim(response.content, 'HTML')
    master_table = extract_tables(content, qtr_date)
    master_table.to_excel(
        writer, sheet_name=qtr_date.replace(',', ''), index=False)
    writer.book.save(path)
writer.close()

In [ ]:
filing_links = filing_links[filing_links['Reporting date']
                            != 'September 30, 2017']

In [ ]:
data_frames = []
data_frames_shapes = []
filing_links = filing_links[filing_links['Reporting date']
                            != 'September 30, 2017']


def extract_tables(soup_content, qtr_date):
    master_table = None
    print(qtr_date)
    if qtr_date == 'December 31, 2015' or qtr_date == 'June 30, 2016':
        consolidated_schedule_regex = re.compile(
            r'(?i)^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*(\(.*\)|)\s*-.*\s*\(.*\)$')
    else:
        consolidated_schedule_regex = re.compile(
            r'(?i)^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    date_regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    # date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'
    for tag in soup_content.find_all(text=re.compile(consolidated_schedule_regex)):
        date_str = re.search(date_regex_pattern, tag.find_next().text)
        if date_str is None:
            date_str = re.search(date_regex_pattern, tag.next.text)
        if date_str is None:
            date_str = re.search(date_regex_pattern,
                                 tag.find_next().next.next.next.text)
        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)
            qtr_date_cleaned = qtr_date.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            date_str_cleaned = date_str.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            print(date_str, qtr_date_cleaned, date_str_cleaned)

            if qtr_date_cleaned == date_str_cleaned:
                html_table = tag.find_next('table')
                new_table = pd.read_html(
                    html_table.prettify(), keep_default_na=False, skiprows=0, flavor='bs4')[0]
                new_table = new_table.applymap(lambda x: unicodedata.normalize(
                    'NFKD', x.strip().strip(u'\u200b').replace('—', '-').replace('%', '')) if type(x) == str else x)

                new_table = new_table.replace(
                    r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                new_table = new_table.dropna(how='all', axis=0)

                if master_table is None:
                    master_table = new_table
                else:
                    master_table = pd.concat(
                        [master_table, new_table], ignore_index=True)
            # print(master_table)

    master_table = master_table.replace('N/A', 'No Value')

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    print(master_table.shape)
    data_frames.append(master_table)
    data_frames_shapes.append(master_table.shape)
    return master_table


# date = "December 31, 2013"
# url = filing_links[filing_links['Reporting date']
#                    == date]['Filings URL'].values[0]
# # webbrowser.open(url=url)
# response = requests.get(url=url, headers=headers)
# content = parse_and_trim(response.content, "HTML")
# master_table = extract_tables(content, date)
# contentDUP = content
# master_tableDUP = master_table


# This does the job
path = '/Users/fuadhassan/Desktop/BDC_RA/GBDC/feedback_Master_tables_GBDC_Investment.xlsx'
if not os.path.exists('../FBMT_csv_file'):
    os.makedirs('../FBMT_csv_file')

writer = pd.ExcelWriter(path, engine='openpyxl')
for qtr_date, html_link in zip(filing_links['Reporting date'], filing_links['Filings URL']):
    print(html_link, qtr_date)
    response = requests.get(html_link, headers=headers)
    content = parse_and_trim(response.content, 'HTML')
    master_table = extract_tables(content, qtr_date)
    master_table.to_excel(
        writer, sheet_name=qtr_date.replace(',', ''), index=False)
    master_table.to_csv(
        '../FBMT_csv_file/'+qtr_date.replace(',', '')+'.csv')
    writer.book .save(path)
writer.close()

In [ ]:
url = 'https://www.sec.gov/Archives/edgar/data/1476765/000147676522000103/gbdc-20220930.htm'
qtr_date = 'September 30, 2022'
response = requests.get(url=url, headers=headers)
content = parse_and_trim(response.content, 'HTML')

In [3]:
# this one was updatde to look for the tables that does not have table title
# there was some missing tables

data_frames = []
data_frames_shapes = []
filing_links = filing_links[filing_links['Reporting date']
                            != 'September 30, 2017']


def extract_tables(soup_content, qtr_date):
    master_table = None
    print(qtr_date)
    if qtr_date == 'December 31, 2015' or qtr_date == 'June 30, 2016':
        consolidated_schedule_regex = re.compile(
            r'(?i)^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*(\(.*\)|)\s*-.*\s*\(.*\)$')
    else:
        consolidated_schedule_regex = re.compile(
            r'(?i)^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')

    date_regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    for tag in soup_content.find_all(text=re.compile(consolidated_schedule_regex)):
        date_str = re.search(date_regex_pattern, tag.find_next().text)
        if date_str is None:
            date_str = re.search(date_regex_pattern, tag.next.text)
        if date_str is None:
            date_str = re.search(date_regex_pattern,
                                 tag.find_next().next.next.next.text)
        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)
            qtr_date_cleaned = qtr_date.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            date_str_cleaned = date_str.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            print(date_str)

            if qtr_date_cleaned == date_str_cleaned:
                html_table = tag.find_next('table')
                new_table = pd.read_html(
                    html_table.prettify(), keep_default_na=False, skiprows=0, flavor='bs4')[0]
                new_table = new_table.applymap(lambda x: unicodedata.normalize(
                    'NFKD', x.strip().strip(u'\u200b').replace('—', '-').replace('%', '')) if type(x) == str else x)

                new_table = new_table.replace(
                    r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                new_table = new_table.dropna(how='all', axis=0)

                if master_table is None:
                    master_table = new_table
                else:
                    master_table = pd.concat(
                        [master_table, new_table], ignore_index=True)

                if datetime.strptime(date_str, '%B %d, %Y').date() > datetime(2022, 3, 30).date():
                    sibling_table = html_table.find_next_sibling('table')
                    if sibling_table:
                        new_table2 = pd.read_html(sibling_table.prettify(
                        ), keep_default_na=False, skiprows=0, flavor='bs4')[0]
                        new_table2 = new_table2.applymap(lambda x: unicodedata.normalize('NFKD', x.strip(
                        ).strip(u'\u200b').replace('—', '-').replace('%', '')) if type(x) == str else x)
                        new_table2 = new_table2.replace(
                            r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                        new_table2 = new_table2.dropna(how='all', axis=0)
                        master_table = pd.concat(
                            [master_table, new_table2], ignore_index=True)

    master_table = master_table.replace('N/A', 'No Value')

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    print(master_table.shape)
    data_frames.append(master_table)
    data_frames_shapes.append(master_table.shape)
    return master_table


# path = '/Users/fuadhassan/Desktop/BDC_RA/GBDC/feedback_Master_tables_GBDC_Investment.xlsx'
# if not os.path.exists('../FBMT_csv_file'):
#     os.makedirs('../FBMT_csv_file')


# writer = pd.ExcelWriter(path, engine='openpyxl')
# for qtr_date, html_link in zip(filing_links['Reporting date'], filing_links['Filings URL']):
#     print(html_link, qtr_date)
#     response = requests.get(html_link, headers=headers)
#     content = parse_and_trim(response.content, 'HTML')
#     master_table = extract_tables(content, qtr_date)
#     master_table.to_excel(
#         writer, sheet_name=qtr_date.replace(',', ''), index=False)
#     master_table.to_csv(
#         '../FBMT_csv_file/'+qtr_date.replace(',', '')+'.csv')
#     writer.book .save(path)
# writer.close()

In [5]:
# this one uses while loop to find the missing ones
filing_links = filing_links[filing_links['Reporting date']
                            != 'September 30, 2017']


def extract_elements_between_text(content, qtr_date):
    master_table = pd.DataFrame()
    consolidated_schedule_regex = re.compile(
        r'(?i)^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    date_regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    start_tag = None
    end_tag = None

    for tag in content.find_all(string=consolidated_schedule_regex):
        date_str = re.search(date_regex_pattern, tag.find_next().text)
        if date_str is None:
            date_str = re.search(date_regex_pattern, tag.next.text)
        if date_str is None:
            date_str = re.search(date_regex_pattern,
                                 tag.find_next().next.next.next.text)

        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)
            qtr_date_cleaned = qtr_date.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            date_str_cleaned = date_str.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            print("Date found")
            if qtr_date_cleaned == date_str_cleaned:
                start_tag = tag
                print("Start_tag found")
                break

    if start_tag:
        for tag2 in content.find_all(string=consolidated_schedule_regex):
            date_str = re.search(date_regex_pattern, tag2.find_next().text)
            if date_str is None:
                date_str = re.search(date_regex_pattern, tag2.next.text)
            if date_str is None:
                date_str = re.search(date_regex_pattern,
                                     tag2.find_next().next.next.next.text)
            if date_str is not None:
                date_str = str(date_str.group(1))
                date_str = unicodedata.normalize('NFKD', date_str)
                qtr_date_cleaned = qtr_date.replace(',', '').replace(
                    ' ', '').replace('\n', '').lower()
                date_str_cleaned = date_str.replace(',', '').replace(
                    ' ', '').replace('\n', '').lower()

                if qtr_date_cleaned != date_str_cleaned:
                    end_tag = tag2
                    print("End_tag found")

                    break

    if start_tag and end_tag:
        elements_html = ""

        current_tag = start_tag.find_parent()
        while current_tag and current_tag != end_tag.find_parent():
            for table in current_tag.find_all('table'):

                elements_html += str(table)
            current_tag = current_tag.find_next()
        if elements_html:
            print("Element in not empty", len(elements_html))
        elements = parse_and_trim(elements_html, 'HTML')

        master_table = None
        for html_table in elements.find_all('table'):

            all_text = html_table.find_all(string=True)

            if not any(text.strip() for text in all_text):
                print("Empty table ignored")
                continue

            new_table = pd.read_html(
                str(html_table), keep_default_na=False, skiprows=0, flavor='bs4')[0]

            new_table = new_table.applymap(lambda x: unicodedata.normalize(
                'NFKD', x.strip().strip(u'\u200b').replace('—', '-').replace('%', '')) if type(x) == str else x)

            new_table = new_table.replace(
                r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)

            new_table = new_table.dropna(how='all', axis=0)

            if master_table is None:
                master_table = new_table

            else:

                master_table = pd.concat(
                    [master_table, new_table], ignore_index=True)

        master_table = master_table.replace('N/A', 'No Value')

        return master_table


path = '/Users/fuadhassan/Desktop/BDC_RA/GBDC/section_tables_GBDC_Investment.xlsx'
if not os.path.exists('../section_MT_csv_file'):
    os.makedirs('../section_MT_csv_file')


writer = pd.ExcelWriter(path, engine='openpyxl')
for qtr_date, html_link in zip(filing_links['Reporting date'], filing_links['Filings URL']):
    print(html_link, qtr_date)
    response = requests.get(html_link, headers=headers)
    content = parse_and_trim(response.content, 'HTML')
    if qtr_date <= pd.to_datetime('2020-01-01'):
        print('extract_tables ran')

        master_table = extract_tables(
            content, qtr_date.strftime("%B %d, %Y"))
    else:
        print('extract_elements_between_text ran')
        master_table = extract_elements_between_text(
            content, qtr_date.strftime("%B %d, %Y"))

    master_table.to_excel(
        writer, sheet_name=qtr_date.strftime("%B %d, %Y").replace(',', ''), index=False)
    master_table.to_csv(
        '../section_MT_csv_file/'+qtr_date.strftime("%B %d, %Y").replace(',', '')+'.csv')
    writer.book .save(path)
writer.close()

https://www.sec.gov/Archives/edgar/data/1476765/000147676524000016/gbdc-20231231.htm 2023-12-31 00:00:00
extract_elements_between_text ran
Date found
Start_tag found
End_tag found
Element in not empty 1177613
https://www.sec.gov/Archives/edgar/data/1476765/000147676523000077/gbdc-20230930.htm 2023-09-30 00:00:00
extract_elements_between_text ran
Date found
Start_tag found
End_tag found
Element in not empty 1188836
https://www.sec.gov/Archives/edgar/data/1476765/000147676523000063/gbdc-20230630.htm 2023-06-30 00:00:00
extract_elements_between_text ran
Date found
Start_tag found
End_tag found
Element in not empty 1091015
Empty table ignored
https://www.sec.gov/Archives/edgar/data/1476765/000147676523000042/gbdc-20230331.htm 2023-03-31 00:00:00
extract_elements_between_text ran
Date found
Start_tag found
End_tag found
Element in not empty 1102179
https://www.sec.gov/Archives/edgar/data/1476765/000147676523000018/gbdc-20221231.htm 2022-12-31 00:00:00
extract_elements_between_text ran
Date 

/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_19482/3544859205.py:21: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for tag in soup_content.find_all(text=re.compile(consolidated_schedule_regex)):


December 31, 2019
December 31, 2019
December 31, 2019
December 31, 2019
December 31, 2019
December 31, 2019
December 31, 2019
December 31, 2019
December 31, 2019
December 31, 2019
December 31, 2019
December 31, 2019
December 31, 2019
December 31, 2019
December 31, 2019
December 31, 2019
December 31, 2019
December 31, 2019
December 31, 2019
December 31, 2019
December 31, 2019
December 31, 2019
December 31, 2019
December 31, 2019
December 31, 2019
December 31, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
(1149, 25)
https://www.sec.gov/Archive

/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_19482/3544859205.py:21: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for tag in soup_content.find_all(text=re.compile(consolidated_schedule_regex)):


September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2019
September 30, 2018
September 30, 2018
September 30, 2018
September 30, 2018
September 30, 2018
September 30, 2018
September 30, 2018
September 30, 2018
September 30, 2018
September 30, 2018
September 30, 2018
September 30, 2018
September 30, 2018
September 30, 2018
September 30, 2018
September 30, 2018
September 30, 2018
September 30, 2018
September 30, 2018
(1096, 25)
https://www.sec.gov/Archives/edgar/data/0001476765/000147676519000064/gbdcfy2019q310-q.htm 2019-06-30 00:00:00
extract_tables ran
June 30, 2019


...

September 30, 2013
September 30, 2013
September 30, 2013
September 30, 2013
September 30, 2013
September 30, 2013
September 30, 2013
September 30, 2013
September 30, 2013
September 30, 2013
September 30, 2013
September 30, 2013
September 30, 2012
September 30, 2012
September 30, 2012
September 30, 2012
September 30, 2012
September 30, 2012
September 30, 2012
September 30, 2012
September 30, 2012
September 30, 2012
September 30, 2012
September 30, 2013
(486, 41)
https://www.sec.gov/Archives/edgar/data/1476765/000114420413043799/v351518_10q.htm 2013-06-30 00:00:00
extract_tables ran
June 30, 2013
June 30, 2013


/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_19482/3544859205.py:21: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for tag in soup_content.find_all(text=re.compile(consolidated_schedule_regex)):


June 30, 2013
June 30, 2013
June 30, 2013
June 30, 2013
June 30, 2013
June 30, 2013
September 30, 2012
September 30, 2012
September 30, 2012
September 30, 2012
September 30, 2012
September 30, 2012
June 30, 2013
(432, 31)
https://www.sec.gov/Archives/edgar/data/1476765/000114420413026113/v343181_10q.htm 2013-03-31 00:00:00
extract_tables ran
March 31, 2013
March
31, 2013


/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_19482/3544859205.py:21: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for tag in soup_content.find_all(text=re.compile(consolidated_schedule_regex)):


March
31, 2013
March
31, 2013
March
31, 2013
March
31, 2013
March
31, 2013
September
30, 2012
September
30, 2012
September
30, 2012
September
30, 2012
September
30, 2012
September
30, 2012
(424, 31)
